I haven't been able to get Jupyter notebooks working yet with the USGS HPCs, but this cluster can't handle the GBIF data.  So I'm trying this manually in Denali, on a single node, to see if I can make progress.

### Import modules

In [5]:
import fsspec
import dask.dataframe as dd
import intake
import os
import sys
import warnings
from dask.distributed import LocalCluster, Client
warnings.filterwarnings('ignore')

#### Start as Dask client

In [ ]:
#On Denali
#from dask.distributed import LocalCluster, Client
#cluster = LocalCluster(threads_per_worker=1)
#client = Client(cluster)
 
#This didn't work.  It spun up the cluster but crashed on the first command.  Let's try it without the cluster.

In [6]:
#On qhub
# sys.path.append(os.path.join(os.environ['HOME'],'shared','users','lib'))
# import ebdpy as ebd
# aws_profile = 'esip-qhub'
# ebd.set_credentials(profile=aws_profile)

# aws_region = 'us-west-2'
# endpoint = f's3.{aws_region}.amazonaws.com'
# ebd.set_credentials(profile=aws_profile, region=aws_region, endpoint=endpoint)
# worker_max = 30
# client,cluster = ebd.start_dask_cluster(profile=aws_profile, worker_max=worker_max, 
#                                         region=aws_region, use_existing_cluster=True,
#                                         adaptive_scaling=False, wait_for_cluster=False, 
#                                         worker_profile='Medium Worker', propagate_env=True)

Region: us-west-2
No Cluster running.
Starting new cluster.
{}
Setting Cluster Environment Variable AWS_DEFAULT_REGION us-west-2
Setting Fixed Scaling workers=30
Reconnect client to clear cache
client.dashboard_link (for new browser tab/window or dashboard searchbar in Jupyterhub):
https://jupyter.qhub.esipfed.org/gateway/clusters/dev.ba22da0d08f74fd99f2884edd0040909/status
Propagating environment variables to workers
Using environment: users/pangeo


_From the above output, copy and paste the url for the daskboard, minus the '/status' into the bar at the top of the dask window on the left of jupyterlab._

It might take 3-5 min for it to spin up.

## Explore dask/data structure

Following tips and tutorials from these:
- https://www.youtube.com/watch?v=6EozEvfTcBI
- https://www.youtube.com/watch?v=LKIRAzsqLb0

In [7]:
#fs = fsspec.filesystem('s3', anon=True)

In [8]:
#fs.ls('s3://gbif-open-data-us-east-1/')

['gbif-open-data-us-east-1/index.html', 'gbif-open-data-us-east-1/occurrence']

In [9]:
#day_list = fs.ls('s3://gbif-open-data-us-east-1/occurrence/')
#day_list[-1]

'gbif-open-data-us-east-1/occurrence/2022-12-01'

In [10]:
#fs.ls(day_list[-1])

['gbif-open-data-us-east-1/occurrence/2022-12-01/citation.txt',
 'gbif-open-data-us-east-1/occurrence/2022-12-01/occurrence.parquet']

In [34]:
# df = dd.read_parquet('s3://gbif-open-data-us-east-1/occurrence/2022-11-01/occurrence.parquet/*', 
#                      storage_options={"anon": True}, 
#                      engine = 'fastparquet')

In [ ]:
#Working locally in Denali

df = dd.read_parquet('/caldera/projects/css/sas/bio/spec_obs/gbif/snapshot/occurrence.parquet/*', 
                      engine = 'fastparquet')

In [35]:
df.head()

,gbifid,datasetkey,occurrenceid,kingdom,phylum,class,order,family,genus,species,...,identifiedby,dateidentified,license,rightsholder,recordedby,typestatus,establishmentmeans,lastinterpreted,mediatype,issue
0,3534344612,4fa7b334-ce0d-4e88-aaae-2e0c138d049e,URN:catalog:CLO:EBIRD:OBS1178111150,Animalia,Chordata,Aves,Passeriformes,Cardinalidae,Passerina,Passerina ciris,...,[],NaT,CC_BY_4_0,None,[obsr455843],[],None,2022-09-08 14:35:29.672,[],[]
1,3534604870,4fa7b334-ce0d-4e88-aaae-2e0c138d049e,URN:catalog:CLO:EBIRD_CAN:OBS1179048592,Animalia,Chordata,Aves,Passeriformes,Emberizidae,Melospiza,Melospiza lincolnii,...,[],NaT,CC_BY_4_0,None,[obsr232608],[],None,2022-09-08 14:35:31.349,[],[]
2,3534885128,4fa7b334-ce0d-4e88-aaae-2e0c138d049e,URN:catalog:CLO:EBIRD:OBS1180074221,Animalia,Chordata,Aves,Passeriformes,Parulidae,Setophaga,Setophaga petechia,...,[],NaT,CC_BY_4_0,None,[obsr313985],[],None,2022-09-08 14:35:34.634,[],[]
3,3535431386,4fa7b334-ce0d-4e88-aaae-2e0c138d049e,URN:catalog:CLO:EBIRD:OBS1180952857,Animalia,Chordata,Aves,Psittaciformes,Psittacidae,Pionus,Pionus menstruus,...,[],NaT,CC_BY_4_0,None,[obsr754360],[],None,2022-09-08 14:35:37.436,[],[]
4,3535679644,4fa7b334-ce0d-4e88-aaae-2e0c138d049e,URN:catalog:CLO:EBIRD:OBS1181682586,Animalia,Chordata,Aves,Piciformes,Picidae,Melanerpes,Melanerpes carolinus,...,[],NaT,CC_BY_4_0,None,[obsr14802],[],None,2022-09-08 14:35:39.348,[],[]


In [36]:
#df.dtypes

## Some thoughts about the data

- Many columns won't be needed.  What's needed are:
    - kingdom
    - class
    - species
    - countrycode
    - decimallatitude
    - decimallongitude
    - year

- decimallatitude and decimal longitude can be converted to int32 to save memory since (I think) most coordinates aren't accurate beyond 4 or 5 decimal places.

Let's check a partition size (in MB). I think it should be aroudn 85 MB, since there are 2021 partitions and the parquet file is ~ 173 GB.


In [14]:
#a single partition
#df.partitions[0].memory_usage(deep=True).compute()/1e6

In [39]:
df.partitions[0].memory_usage(deep=True).sum().compute()/1e6

3457.245578

Hmm, that returns 3457 MB. Let me try something else. Strange, none of them are very even, and they are all kinds of sizes. The internet tells me "For example if your dask.dataframe has only one partition then only one core can operate at a time. If you have too many partitions then the scheduler may incur a lot of overhead deciding where to compute each task. Generally you want a few times more partitions than you have cores."

In [42]:
part_list = df.memory_usage_per_partition(deep=True).compute() / 1e6

In [45]:
part_sum = part_list.sum()
part_sum / 1e3 #in GB. So 4.7TB. 

4762.149198174

In [46]:
part_mean = part_list.mean()
part_mean #2.3 GB

2356.333101521029

What would be an even split?

In [51]:
import math
math.sqrt(part_sum) # 2.18 GB, with 2182 partitions

2182.234909026524

## More thoughts...

- not clear if any of these are horrible, but from the tutorial, it's likely that the objects should be converted to "string[pyarrow]"  Not too suprisingly, these are the most expensive.
- After thinking about it some more, it's probably not safe to convert float 64 to float32. Too much precision would be lost.
- Let's check total memory volume 

In [15]:
#import dask

#dask.utils.format_bytes(
#    df.memory_usage(deep=True).compute().sum()
#)

_That took a little while, not sure if it was worth it. It says it's 4.33 TiB, that's much larger than I thought it would be._

In [53]:
conversions = {}
for column, dtype in df.dtypes.items():
    if dtype == "object":
        conversions[column] = "string[pyarrow]"
        
conversions

{'datasetkey': 'string[pyarrow]',
 'occurrenceid': 'string[pyarrow]',
 'kingdom': 'string[pyarrow]',
 'phylum': 'string[pyarrow]',
 'class': 'string[pyarrow]',
 'order': 'string[pyarrow]',
 'family': 'string[pyarrow]',
 'genus': 'string[pyarrow]',
 'species': 'string[pyarrow]',
 'infraspecificepithet': 'string[pyarrow]',
 'taxonrank': 'string[pyarrow]',
 'scientificname': 'string[pyarrow]',
 'verbatimscientificname': 'string[pyarrow]',
 'verbatimscientificnameauthorship': 'string[pyarrow]',
 'countrycode': 'string[pyarrow]',
 'locality': 'string[pyarrow]',
 'stateprovince': 'string[pyarrow]',
 'occurrencestatus': 'string[pyarrow]',
 'publishingorgkey': 'string[pyarrow]',
 'basisofrecord': 'string[pyarrow]',
 'institutioncode': 'string[pyarrow]',
 'collectioncode': 'string[pyarrow]',
 'catalognumber': 'string[pyarrow]',
 'recordnumber': 'string[pyarrow]',
 'identifiedby': 'string[pyarrow]',
 'license': 'string[pyarrow]',
 'rightsholder': 'string[pyarrow]',
 'recordedby': 'string[pyarrow

In [54]:
df = df.astype(conversions)

**How big is the first partition?**

In [55]:
#df.partitions[0].memory_usage(deep=True).compute()/1000000

Wow, that was significant for the strings.

## Repartition to recommended size
...he talks about this in the video, I tried 256 MiB. Didn't seem to make much of a difference.  I need to read up on this more.

In [56]:
#df = df.repartition(partition_size="256MiB").persist()

## Further filtering...

From this data quality presentation, there are some recommendations about flags and filters to use.

- zero-zero coordinates should be removed (although if we're just thinking about USA this isn't a problem)
- country coordinate mismatch (under issues) can be used to filter to USA, and otherwise remove spurious data.
- remove absence records
- remove coordinate invalid records
- remove coordinate out of range records
- remove fossil records
- remove living specimen records (in zoo, botanical gardens)
- count and (maybe) remove data in country centroids

*Select the columns we need for filtering. First examine values*

Some memory is saved by converting issue, basisofrecord, kingdom, and countrycode to category...but according to Ayush, there's a known bug with categorical columns when using to_parquet to save the dask dataframe into parquet files. He says, "It affects some environments (at least it did mine) while transforming the encoding."  So we'll avoid back convert these when writing to parquet.

In [57]:
df['issue'] = df.issue.astype('category')
df['basisofrecord'] = df.basisofrecord.astype('category')
df['countrycode'] = df.countrycode.astype('category')
df['kingdom'] = df.kingdom.astype('category')

#df.partitions[0].memory_usage(deep=True).compute()/1000000

## Find levels

Because dask [splits categoricals](https://docs.dask.org/en/stable/dataframe-categoricals.html), if we want to know all possible levels, we need to make them "known" first...but this requires scan of the entire data, so not really worth it.  I got the levels by going to GBIF.org and looking at the URL after filtering these with the search tool.

In [58]:
#df = df.categorize(columns= ['issue', 'basisofrecord'])

In [ ]:
df1 = df[["issue",
         "basisofrecord",
         "occurrencestatus",
         "kingdom",
         "class",
         "countrycode",
         "decimallongitude",
         "decimallatitude",
         "year",
         "species"]]

In [64]:
df1 = df1[~df1['issue'].isin(["['COUNTRY_COORDINATE_MISMATCH']",
                             "['RECORDED_DATE_INVALID']",
                             "['COORDINATE_INVALID']"])
         ]

In [65]:
df1 = df1[df1['occurrencestatus'] == "PRESENT"]

In [66]:
df1 = df1[df1['basisofrecord'].isin(["OBSERVATION",
                                  "MACHINE_OBSERVATION",
                                  "HUMAN_OBSERVATION",
                                  "MATERIAL_SAMPLE",
                                  "PRESERVED_SPECIMEN",
                                  "OCCURRENCE"])
       ]

Let's subset to USA. I learned here that it's necessary to use double quotes aroudn the US.

In [67]:
df1 = df1[df1.countrycode == "US"]

filter to 2010

In [68]:
df1 = df1[df1.year == 2010]
df1 = df1[df1.year <= 2019]

Remove unneeded columns

In [69]:
df1 = df1[["kingdom",
         "class",
         "decimallongitude",
         "decimallatitude",
         "year",
         "species"]]

In [70]:
df1.head(n = 2)

,kingdom,class,decimallongitude,decimallatitude,year,species
170,Animalia,Aves,-89.28571,38.666687,2010,Buteo jamaicensis
171,Animalia,Aves,-155.32988,19.644060,2010,Lophura leucomelanos


After subsetting and repartitioning, the volume in memory is 

In [71]:
df1.partitions[0].memory_usage(deep=True).sum().compute()/1e6

0.675021

Ok, so the one got much smaller.

In [73]:
part_list = df1.memory_usage_per_partition(deep=True).compute() / 1e6

In [77]:
part_sum = part_list.sum()
part_sum # 1GB. 

969.190003

In [75]:
part_mean = part_list.mean()
part_mean #0.5 MB

0.47955962543295405

What would be an even split?

In [76]:
import math
math.sqrt(part_sum) # 31 MB with 31 partitions, but we can handle more than that.

31.131816570833127

Drop NA values in species (not totally sure we should drop these as much as aggregate them at genus, but that's a discussion for another day.) and decimallatitude (could use longtitude instead). _Follow up: I think this dropna is really computationally expensive (from watching the workers) but I'm not sure why.  Something to research._

In [78]:
#df1 = df1.repartition(partition_size="100MB")

In [82]:
df1 = df1.persist()

2022-12-08 23:57:55,063 - distributed.client - ERROR - Exception raised while shutting down cluster dev.ba22da0d08f74fd99f2884edd0040909
Traceback (most recent call last):
  File "/home/conda/users/7d479acd42e90471353def6ca5a8af746b5fc7262141e86e2738676e7087dbc8-20221121-135211-565707-232-pangeo/lib/python3.9/site-packages/dask_gateway/client.py", line 993, in _stop_internal
    await self.gateway._stop_cluster(self.name)
  File "/home/conda/users/7d479acd42e90471353def6ca5a8af746b5fc7262141e86e2738676e7087dbc8-20221121-135211-565707-232-pangeo/lib/python3.9/site-packages/dask_gateway/client.py", line 654, in _stop_cluster
    await self._request("DELETE", url)
  File "/home/conda/users/7d479acd42e90471353def6ca5a8af746b5fc7262141e86e2738676e7087dbc8-20221121-135211-565707-232-pangeo/lib/python3.9/site-packages/dask_gateway/client.py", line 397, in _request
    resp = await session.request(method, url, json=json, **self._request_kwargs)
  File "/home/conda/users/7d479acd42e90471353def6

_The above started getting unwieldy with the whole datset, with a lot written to disk.  Repartitioning didn't help and it crashed the cluster. (or at least threw an error about shuting down the cluster)It seems to be able to handle the US 2010s though.

_It may have helped a little.  There is still a fair amount on disk (in grey) but I think that's going to have to be the way it is because of the size of the cluster.  Check volume below._

In [ ]:
#On Denali, write parquet, just to see.  Keep in mind what Anush said about categoricals.

dd.to_parquet(df1, '/caldera/projects/css/sas/bio/spec_obs/gbif/snapshot/US_2010s_occurrence.parquet/*')


In [ ]:
#df2 = df1.groupby(["year", "decimallongitude","decimallatitude"]).count().persist()

The cluster crashed a few times trying to compute the above.  It got most of the way there, then was killed and started over. It did this three times and then gave up.

In [33]:
#del df1

## Mess around with datashader

followed this: https://www.youtube.com/watch?v=LKIRAzsqLb0

In [ ]:

# import datashader
# from datashader import transfer_functions as tf
# from datashader.colors import Hot
# import holoviews as hv

# def render(df1, x_range=(-125, -65), y_range=(24, 50)):
#     # Plot
#     canvas = datashader.Canvas(
#         x_range=x_range,
#         y_range=y_range,
#     )
#     agg = canvas.points(
#         source=df, 
#         x="decimallongitude", 
#         y="decimallatitude", 
#         agg=datashader.count("kingdom"),
#     )
#     return datashader.transfer_functions.shade(agg, cmap=Hot, how="eq_hist")

In [ ]:
#render(df1)

## List your clusters, then shut them all down

In [5]:
from dask_gateway import Gateway
from dask.distributed import Client
gateway = Gateway()

In [6]:
gateway.list_clusters()

[]

In [3]:
for cluster in gateway.list_clusters():
    gateway.stop_cluster(cluster_name=cluster.name)
    print(f'stopped {cluster.name}')

stopped dev.ba22da0d08f74fd99f2884edd0040909


In [4]:
gateway.list_clusters()

[]